# read files

In [1]:
import pymssql
import pymysql as mysql
import os
import re
import pandas as pd
import datetime

In [2]:
class stt_server:

    def __init__(self, cpu_id):

        # settings ++
        self.cpu_id = cpu_id
        self.cpu_cores = [i for i in range(0,15)]

        # ms sql
        self.sql_name = 'voice_ai'
        self.sql_server = '10.2.4.124'
        self.sql_login = 'ICECORP\\1c_sql'

        # mysql
        self.mysql_name = {
            1: 'MICO_96',
            2: 'asterisk',
        }
        self.mysql_server = '10.2.4.146'
        self.mysql_login = 'asterisk'

        self.script_path = '/home/alex/projects/call_centre_stt_server/'
        self.model_path = '/home/alex/projects/vosk-api/python/example/model'
        self.source_id = 0
        self.sources = {
            'call': 1,
            'master': 2,
        }
        self.original_storage_path = {
            1: '/mnt/share/audio_call/',
            2: '/mnt/share/audio_master/REC_IN_OUT/',
        }
        self.original_storage_prefix = {
            1: 'RXTX_',
            2: '',
        }
        self.temp_file_path = self.script_path+'files/'
        # settings --

        self.temp_file_name = ''
        self.original_file_path = ''
        self.original_file_name = ''
        self.original_file_duration	= 0
        self.date_y = ''
        self.date_m = ''
        self.date_d = ''
        self.rec_date = ''

        #store pass in file, to prevent pass publication on gitdelete_current_queue
        with open(self.script_path+'sql.pass','r') as file:
            self.sql_pass = file.read().replace('\n', '')
            file.close()

        with open(self.script_path+'mysql.pass','r') as file:
            self.mysql_pass = file.read().replace('\n', '')
            file.close()

        self.conn = self.connect_sql()
        self.mysql_conn = {
            1: self.connect_mysql(1),
            2: self.connect_mysql(2),
        }

    def connect_sql(self):

        return pymssql.connect(
            server = self.sql_server,
            user = self.sql_login,
            password = self.sql_pass,
            database = self.sql_name,
            #autocommit=True
        )

    def connect_mysql(self, source_id):

        return mysql.connect(
            host = self.mysql_server, 
            user = self.mysql_login, 
            passwd = self.mysql_pass,
            db = self.mysql_name[source_id],
            #autocommit = True
        )

    def get_fs_files_list(self):

        self.original_file_path = self.original_storage_path[self.source_id]
        self.original_file_path += self.original_storage_prefix[self.source_id]
        if self.source_id == self.sources['call']:
            self.original_file_path += self.date_y + '-' + self.date_m + '/' + self.date_d + '/'
        files_list = []
        for (dirpath, dirnames, filenames) in os.walk(self.original_file_path):
            files_list.extend(filenames)
            break

        # get record date
        fd_list = []
        for filename in files_list:

            rec_date = 'Null'

            if self.source_id == self.sources['call']:
                rec_source_date = re.findall(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', filename)[0]
                if len(rec_source_date):
                    rec_date = rec_source_date[:10] + ' ' + rec_source_date[11:].replace('-', ':')
                if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', rec_date)) == 0:
                    rec_date = 'Null'
                    print('Unable to extract date from filename', filename)

            elif self.source_id == self.sources['master']:
                uniqueid = re.findall(r'^\d*.\d*', filename)[0]
                cursor = self.mysql_conn[self.source_id].cursor()
                query = "select calldate from cdr where uniqueid = '" + uniqueid + "' limit 1;"
                cursor.execute(query)
                for row in cursor.fetchall():
                    rec_date = str(row[0])

            fd_list.append({
                'filename': filename,
                'rec_date': rec_date
            })

            #break  # todo: REMOVE

        df = pd.DataFrame(fd_list)
        df.sort_values(['rec_date', 'filename'], ascending=True, inplace=True)

        return df.values
    
    def get_sql_complete_files(self):

        cursor = self.conn.cursor()
        
        if self.source_id == self.sources['call']:
            sql_query = "select distinct filename from queue where "
            sql_query += "source_id='"+str(self.source_id)+"' and "
            sql_query += "date_y='"+self.date_y+"' and "
            sql_query += "date_m='"+self.date_m+"' and "
            sql_query += "date_d='"+self.date_d+"' "
            sql_query += "union all "
            sql_query += "select distinct audio_file_name from transcribations where "
            sql_query += "date_y='"+self.date_y+"' and "
            sql_query += "date_m='"+self.date_m+"' and "
            sql_query += "date_d='"+self.date_d+"' "
            sql_query += "order by filename;"
        
        elif self.source_id == self.sources['master']:
            sql_query = "select distinct filename from queue "
            sql_query += "where source_id='"+str(self.source_id)+"' "
            sql_query += "order by filename;"
            
        cursor.execute(sql_query)
        complete_files = []
        for row in cursor.fetchall():
            complete_files.append(row[0])

        return complete_files
    
    def get_source_id(self, source_name):
        for source in self.sources.items():
            if source[0] == source_name:
                return source[1]
        return 0
    
    def set_today_ymd(self):
        self.date_y	= datetime.datetime.today().strftime('%Y')
        self.date_m	= datetime.datetime.today().strftime('%m')
        self.date_d	= datetime.datetime.today().strftime('%d')

In [ ]:
server_object = stt_server(0)
server_object.set_today_ymd()
for source_id in server_object.sources: # ['call', 'master']
    #server_object.source_id = server_object.sources['call']
    server_object.source_id = server_object.get_source_id(source_id)
    complete_files	= server_object.get_sql_complete_files()
    incomplete_count = 0
    complete_count = 0
    print('server_object.source_id', server_object.source_id)
    for filename, rec_date in server_object.get_fs_files_list():
        print(filename, rec_date)
    break

In [ ]:
complete_files

In [ ]:
server_object = stt_server(0)
server_object.source_id = 2
fd_list = server_object.get_fs_files_list()
fd_list

In [ ]:
df = pd.DataFrame(fd_list)
df.sort_values(['rec_date', 'filename'], ascending=False, inplace=True)

In [ ]:
#df[df.rec_date=='Null']
for f,d in df.values:
    print(f,d)
    break

In [ ]:
filename = '1612946589.1564912-in.wav'
uniqueid = re.findall(r'^\d*.\d*', filename)[0]
uniqueid

In [44]:
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "select calldate from cdr where uniqueid = '"+uniqueid+"' limit 1;"
cursor.execute(query)
for row in cursor.fetchall():
    #rec_date = str(row[0])
    print(row[0])

2021-02-04 11:36:33


In [ ]:
def get_source_id(source_name):
    for source in server_object.sources.items():
        if source[0] == source_name:
            return source[1]
    return 0
get_source_id('master')

In [ ]:
server_object.rec_date = 'Null'
filename = 'in_4957237230_2021-02-10-07-16-03rxtx.wav'
rec_source_date = re.findall(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', filename)[0]                
if len(rec_source_date):
    server_object.rec_date = rec_source_date[:10] + ' ' + rec_source_date[11:].replace('-', ':')
if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', server_object.rec_date)) == 0:
    print('Unable to extract date from filename', filename)
server_object.rec_date

In [ ]:
rec_source_date

In [ ]:
server_object.rec_date

In [ ]:
re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', server_object.rec_date)

In [ ]:
for source_id in server_object.sources:
    print(get_source_id(source_id))

In [ ]:
server_object = stt_server(0)
server_object.source_id = 1

In [ ]:
files = server_object.get_fs_files_list()
len(files), files[0]

In [ ]:
files

In [ ]:
complete = server_object.get_sql_complete_files()
len(complete)

### transcribations show columns

In [37]:
cursor = server_object.conn.cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('transcribation_date',)
('date_y',)
('date_m',)
('date_d',)
('side',)
('text',)
('start',)
('audio_file_name',)
('conf',)
('end_time',)
('sentiment',)
('sentiment_pos',)
('sentiment_neg',)
('ID',)
('linkedid',)
('dst',)
('record_date',)
('source_id',)
('src',)


### check date cdr

In [48]:
datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2021-02-11 12:16:12'

In [46]:
uniqueid = '1613032036.1593933'

In [47]:
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "select calldate from cdr where uniqueid = '"+uniqueid+"' limit 1;"
cursor.execute(query)
for row in cursor.fetchall():
    #rec_date = str(row[0])
    print(row[0])

2021-02-11 11:27:16


### PT1C_cdr_MICO show columns

In [14]:
cursor = server_object.mysql_conn[1].cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='PT1C_cdr_MICO';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('id',)
('calldate',)
('clid',)
('src',)
('dst',)
('dcontext',)
('lastapp',)
('lastdata',)
('duration',)
('billsec',)
('disposition',)
('channel',)
('dstchannel',)
('amaflags',)
('accountcode',)
('uniqueid',)
('userfield',)
('answer',)
('end',)
('recordingfile',)
('peeraccount',)
('linkedid',)
('sequence',)


### queue show columns

In [9]:
cursor = server_object.conn.cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='queue';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('date',)
('cpu_id',)
('filepath',)
('date_y',)
('date_m',)
('date_d',)
('filename',)
('duration',)
('record_date',)
('source_id',)


### transcribations show columns

In [12]:
cursor = server_object.conn.cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('transcribation_date',)
('date_y',)
('date_m',)
('date_d',)
('side',)
('text',)
('start',)
('audio_file_name',)
('conf',)
('end_time',)
('sentiment',)
('sentiment_pos',)
('sentiment_neg',)
('ID',)
('linkedid',)
('dst',)
('record_date',)
('source_id',)
('src',)


### get record date from cdr

In [6]:
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='cdr';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('calldate',)
('clid',)
('src',)
('dst',)
('dcontext',)
('channel',)
('dstchannel',)
('lastapp',)
('lastdata',)
('duration',)
('billsec',)
('disposition',)
('amaflags',)
('accountcode',)
('uniqueid',)
('userfield',)
('did',)
('recordingfile',)
('calldate',)
('clid',)
('src',)
('dst',)
('dcontext',)
('channel',)
('dstchannel',)
('lastapp',)
('lastdata',)
('duration',)
('billsec',)
('disposition',)
('amaflags',)
('accountcode',)
('uniqueid',)
('userfield',)
('did',)
('recordingfile',)
('calldate',)
('clid',)
('src',)
('dst',)
('dcontext',)
('channel',)
('dstchannel',)
('lastapp',)
('lastdata',)
('duration',)
('billsec',)
('disposition',)
('amaflags',)
('accountcode',)
('uniqueid',)
('userfield',)
('did',)
('recordingfile',)
('id',)
('calldate',)
('clid',)
('src',)
('dst',)
('dcontext',)
('channel',)
('dstchannel',)
('lastapp',)
('lastdata',)
('duration',)
('billsec',)
('disposition',)
('amaflags',)
('accountcode',)
('uniqueid',)
('peeraccount',)
('linkedid',)
('sequence',)
('userfield',)

In [12]:
server_object.source_id = 2
#server_object.original_file_name = '1612427793.1440877-in.wav'#files[0] #debug
server_object.original_file_name = '1612427793.1440877-in.wav'#files[0] #debug
uniqueid = re.findall(r'^\d*.\d*', server_object.original_file_name)[0]
print('uniqueid', uniqueid)
cursor = server_object.mysql_conn[server_object.source_id].cursor()
#query = "select src, dst from cdr where uniqueid = '"+uniqueid+"' limit 1;"
query = "select calldate, src, dst from cdr where uniqueid = '"+uniqueid+"' limit 1;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

uniqueid 1612427793.1440877
(datetime.datetime(2021, 2, 4, 11, 36, 33), '5113', '989533444853')


### insert into queue debug

In [55]:
server_object = stt_server(0)
sql_query = "insert into queue (filepath, filename, cpu_id, date, date_y, date_m, date_d, duration, record_date, source_id, src, dst, linkedid) values ('/mnt/share/audio_master/REC_IN_OUT/','1613043347.1600548-out.wav','3','2021-02-11 14:36:41','2021','02','11','0',Null,'2','1810','989859826939','1613042947.1600204');"
cursor = server_object.conn.cursor()
cursor.execute(sql_query)
server_object.conn.commit() # autocommit

### delete from queue

In [50]:
server_object = stt_server(0)
#server_object.source_id = 1
cursor = server_object.conn.cursor()
sql_query = "delete from queue;"
cursor.execute(sql_query)
server_object.conn.commit() # autocommit

### select from queue

In [53]:
server_object = stt_server(0)
server_object.source_id = 2

cursor = server_object.conn.cursor()
#sql_query = "select count(filename) from queue where source_id = '"+str(server_object.source_id)+"';"
#sql_query = "select distinct cpu_id from queue where source_id = '"+str(server_object.source_id)+"';"
#sql_query = "select filename, record_date, cpu_id, duration, source_id from queue "
#sql_query += "where source_id='" + str(server_object.source_id) + "' "
#sql_query += "where duration=60 "
#sql_query += "order by record_date;"

print('# 1')
sql_query = "select top 1 * from queue "
sql_query += "where source_id='1';"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)

print('# 2')
sql_query = "select top 1 * from queue "
sql_query += "where source_id='2';"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)
    
print('# count 1')
sql_query = "select count(filename) from queue where source_id = '1';"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)
    
print('# count 2')
sql_query = "select count(filename) from queue where source_id = '2';"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)
    
print('# cpus')
sql_query = "select distinct cpu_id from queue where cpu_id='30' order by cpu_id;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)


# 1
(datetime.datetime(2021, 2, 11, 12, 18, 22), 11, '/mnt/share/audio_call/RXTX_2021-02/11/', '2021', '02', '11', 'in_4957237230_2021-02-11-07-16-05rxtx.wav', 8.3, datetime.datetime(2021, 2, 11, 7, 16, 5), 1, '79057407030', 'None', '1612992225.1482483')
# 2
(datetime.datetime(2021, 2, 11, 12, 18, 55), 17, '/mnt/share/audio_master/REC_IN_OUT/', '2021', '02', '11', '1612425037.1439526-in.wav', 0.0, datetime.datetime(2021, 2, 4, 10, 50, 37), 2, '9055316577', '989168860883', '1612425037.1439526')
# count 1
(1260,)
# count 2
(59069,)
# cpus
(30,)


### delete from transcribations

In [49]:
cursor = server_object.conn.cursor()
sql_query = "delete from transcribations where source_id = '2';"
cursor.execute(sql_query)
server_object.conn.commit() # autocommit

### transcribed phrases

In [52]:
server_object = stt_server(0)
server_object.source_id = 2

cursor = server_object.conn.cursor()

sql_query = "select audio_file_name, record_date, start, side, text from transcribations where source_id = '2' order by record_date, start, side;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

('1612427076.1440458-in.wav', datetime.datetime(2021, 2, 4, 11, 24, 36), 2.1, False, 'а')
('1612427076.1440458-in.wav', datetime.datetime(2021, 2, 4, 11, 24, 36), 17.85, False, 'здравствуйте сервисный центр по поводу холодильником')
('1612427076.1440458-in.wav', datetime.datetime(2021, 2, 4, 11, 24, 36), 22.2, False, 'меня зовут евгений что у вас случилось')
('1612427076.1440458-in.wav', datetime.datetime(2021, 2, 4, 11, 24, 36), 35.01, False, 'но насколько сильно порвалась магнита высыпались на месте')
('1612427076.1440458-in.wav', datetime.datetime(2021, 2, 4, 11, 24, 36), 44.67, False, 'ну разберу такие сквозные и несквозные')
('1612427076.1440458-in.wav', datetime.datetime(2021, 2, 4, 11, 24, 36), 63.36, False, 'ну не совсем понял что значит до конца резина она многослойная вас все слои спор')
('1612427076.1440458-in.wav', datetime.datetime(2021, 2, 4, 11, 24, 36), 85.98, False, 'но вдруг один слой верхние и останется своровал работают')
('1612427076.1440458-in.wav', datetime.datet

In [18]:
sql_query

"select * from queue where source_id='2' limit 1;"

In [ ]:
server_object.get_sql_complete_files()

### select from transcribations

In [4]:
server_object = stt_server(0)
#server_object.source_id = 1

cursor = server_object.conn.cursor()
sql_query = "select top 1 * from transcribations;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

(datetime.datetime(2020, 10, 9, 8, 39, 8), '2020', '10', '09', True, '', 0.0, 'in_4957237230_2020-10-09-07-36-05rxtx.wav', None, None, None, None, None, 1, None, None, datetime.datetime(2020, 10, 9, 7, 36, 5), 1)


# other

In [ ]:
import datetime

In [ ]:
cur_date = datetime.datetime.now()
DD = datetime.timedelta(days=int(365 / 2))
crop_date = cur_date - DD
cur_date_y = crop_date.strftime("%Y")
cur_date_m = crop_date.strftime("%m")
cur_date_d = crop_date.strftime("%d")
cur_date_y, cur_date_m, cur_date_d

In [ ]:
cur_date = datetime.datetime.now()
cur_date

In [ ]:
DD = datetime.timedelta(days=int(365/2))
crop_date = cur_date - DD
crop_date

In [ ]:
#datetime.datetime.now().year
datetime.datetime.now().strftime("%Y")

In [ ]:
#datetime.datetime.now().month
datetime.datetime.now().strftime("%m")

In [ ]:
cur_date = datetime.datetime.strptime("2021-04-12T07:00:00Z","%Y-%m-%dT%H:%M:%SZ")
DD = datetime.timedelta(days=int(365 / 2))
crop_date = cur_date - DD
#datetime.datetime.strptime(str(datetime.datetime.now()),"%m")
crop_date

In [ ]:
#datetime.datetime.now().day
datetime.datetime.now().strftime("%d")

In [ ]:
import pandas as pd
from init_server import stt_server
from deeppavlov import build_model, configs
import numpy as np

In [ ]:
BATCH_SIZE = 3

In [ ]:
server_object = stt_server(0)

In [ ]:
query = """
    select top """+str(BATCH_SIZE)+""" 
    id,
    text,
    sentiment
    from transcribations 
    where sentiment is NULL and text!=''
    """

In [ ]:
df = pd.read_sql(query, server_object.conn)

model = build_model(configs.classifiers.rusentiment_bert, download=True) #download first time
res = model(df.text)
df['sentiment'] = model(df.text)

In [ ]:
print(df)

In [ ]:
df['sentiment'] = ['neutral','negative','skip']

In [ ]:
df

In [ ]:
for index, row in df.iterrows():
    print(index, row.id,row.sentiment)

In [ ]:
for row in df.values:
    print(row.sentiment)

In [ ]:
for index, row in df.iterrows():
    if row.sentiment == 'negative':
        neg = 1
        pos = 0
    else:
        neg = 0
        pos = 1
    query = "update transcribations set "
    query += "sentiment = '"+row.sentiment+"', "
    query += "sentiment_neg = "+str(neg)+", "
    query += "sentiment_pos = "+str(pos)+" "
    query += "where id = "+str(row.id)
    break

In [ ]:
query

In [ ]:
df['sentiment_neg']=np.zeros(len(df))
df['sentiment_pos']=np.zeros(len(df))

In [ ]:
sentiments = pd.DataFrame(columns=['sentiment','sentiment_neg','sentiment_pos'])

In [ ]:
sentiments = sentiments.append(pd.DataFrame({
    'sentiment':
        [
            'negative',
            'positive',
            'neutral',
            'speech',
            'skip'
        ],
    'sentiment_neg':
        [
            1,0,0,0,0
        ],
    'sentiment_pos':
        [
            0,1,1,1,1
        ]
}))

In [ ]:
sentiments

In [ ]:
df

In [ ]:
df[df.sentiment=='negative'].sentiment_neg=np.ones(len(df[df.sentiment=='negative']))

In [ ]:
df[df.sentiment=='negative']

In [ ]:
df[df.sentiment=='negative'].sentiment_neg=[1]

In [ ]:
df[df.sentiment=='negative'].sentiment_neg

In [ ]:
df.set_value[df.sentiment=='negative']['sentiment_neg']=np.ones(len(df[df.sentiment=='negative']))
#df[df.sentiment!='negative']['sentiment_pos']=1
#df.fillna(0)
#df.set_value('C', 'x', 10)

In [ ]:
df[df.sentiment=='negative']['sentiment_neg']